<a href="https://colab.research.google.com/github/Myavuzars/Text-Recognition-w-Tensorflow/blob/main/Sentiment_Analysis_Amazon_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np
import pandas as pd

In [2]:
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

--2021-03-27 16:55:41--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 172.217.9.206, 2607:f8b0:4004:806::200e
Connecting to drive.google.com (drive.google.com)|172.217.9.206|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/e1v103bmfeq06u83rfv2ubl3h251guis/1616864100000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P [following]
--2021-03-27 16:55:41--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/e1v103bmfeq06u83rfv2ubl3h251guis/1616864100000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 172.217.7.129, 2607:f8b0:4004:805::2001
Connecting to doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)|172.217.7.129|:443

In [3]:
dataset = pd.read_csv('/tmp/sentiment.csv')
dataset.head(10)

,Unnamed: 0,text,sentiment
0,0,So there is no way for me to plug it in here i...,0
1,1,Good case Excellent value.,1
2,2,Great for the jawbone.,1
3,3,Tied to charger for conversations lasting more...,0
4,4,The mic is great.,1
5,5,I have to jiggle the plug to get it to line up...,0
6,6,If you have several dozen or several hundred c...,0
7,7,If you are Razr owner...you must have this!,1
8,8,Needless to say I wasted my money.,0
9,9,What a waste of money and time!.,0


In [4]:
sentences = dataset['text'].tolist()
labels= dataset['sentiment'].tolist()

In [5]:
print(len(labels))

1992


In [6]:
#Data Preparation
size= int(len(labels)*0.8)

training_sentences =sentences[:size]
training_labels = labels[:size]

test_sentences = sentences[size:]
test_labels = labels[size:]

#Make labels np array
test_labels_array = np.array(test_labels)
train_labels_array =np.array(training_labels)


In [7]:
vocab_size= 1000
embedding_dim = 16
max_length = 100
trunc_type ='post'
padding_type= 'post'
oov_tok = "<OOV>"



In [8]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded= pad_sequences(sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)

testing_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded= pad_sequences(testing_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)


In [9]:
#Review a Sequence
reverse_word_index =dict([(value,key) for (key,value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i,'.') for i in text])

print(decode_review(training_padded[1]))
print(training_sentences[1])

good case excellent value . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Good case Excellent value.


In [ ]:
#Building a model

model= tf.keras.Sequential([
                            tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(6,activation='relu'),
                            tf.keras.layers.Dense(1, activation='sigmoid')

])

model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])

model.summary()


In [27]:
num_epohs =10

model.fit(training_padded,train_labels_array, epochs=num_epohs, validation_data=(testing_padded, test_labels_array) )

Epoch 1/10
50/50 [==============================] - 1s 7ms/step - loss: 0.6930 - accuracy: 0.5234 - val_loss: 0.6995 - val_accuracy: 0.4110
Epoch 2/10
50/50 [==============================] - 0s 2ms/step - loss: 0.6839 - accuracy: 0.5222 - val_loss: 0.6982 - val_accuracy: 0.4185
Epoch 3/10
50/50 [==============================] - 0s 3ms/step - loss: 0.6606 - accuracy: 0.5581 - val_loss: 0.6600 - val_accuracy: 0.6416
Epoch 4/10
50/50 [==============================] - 0s 3ms/step - loss: 0.5711 - accuracy: 0.8311 - val_loss: 0.5747 - val_accuracy: 0.7243
Epoch 5/10
50/50 [==============================] - 0s 3ms/step - loss: 0.4285 - accuracy: 0.8939 - val_loss: 0.5143 - val_accuracy: 0.7594
Epoch 6/10
50/50 [==============================] - 0s 3ms/step - loss: 0.3143 - accuracy: 0.9246 - val_loss: 0.4946 - val_accuracy: 0.7519
Epoch 7/10
50/50 [==============================] - 0s 3ms/step - loss: 0.2220 - accuracy: 0.9502 - val_loss: 0.4675 - val_accuracy: 0.7769
Epoch 8/10
50/50 [==

In [11]:
#Prdeciting Sentiment analysis
fake_reviews= ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

In [12]:
# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')

# Try adding reviews of your own
# Add some negative words (such as "not") to the good reviews and see what happens
# For example:
# they gave us free chocolate cake and did not charge us

I love this phone
[0.49272156]


I hate spaghetti
[0.49470723]


Everything was cold
[0.49762055]


Everything was hot exactly as I wanted
[0.49395165]


Everything was green
[0.4953017]


the host seated us immediately
[0.49443552]


they gave us free chocolate cake
[0.4927144]


not sure about the wilted flowers on the table
[0.49386147]


only works when I stand on tippy toes
[0.493145]


does not work when I stand on my head
[0.49414265]


